In [2]:
#helper functions
import math
from sympy.ntheory import factorint
import numpy as np
from progressbar import *    

def gcd(x, y): 
    while(y): 
        x, y = y, x % y 
    return x

def prime(x): #TODO - improve
    #if 
    for i in range(2, round(math.sqrt(x)) + 1):
        if x % i == 0:
            return False
    return True


def add_cols(M, a, b):
    
    return np.array([(M[i][a] + M[i][b]) % 2 for i in range(len(M))])

def find_pivot(M, col):
    for i, row in enumerate(range(len(M))):
        if M[row][col] == 1: return i
    
    return None


def reduce_matrix(M):
    #taken from https://www.sciencedirect.com/science/article/pii/074373159190115P
    #Assume M is K+1 x K
    
    ##Gaussian elimination of matrix
    K = len(M[0])
    
    marked = [0] * (K+1)
    pivots = [None] * K

    for j in range(K):
        i = find_pivot(M, j)
        if i is not None:
            marked[i] = True
            pivots[j] = i
            for k in range(K):
                if M[i][k] == 1 and j != k:
                    #print("k=", k)
                    M[:, k] = add_cols(M, j, k)
    
    #finding dependent rows
    rows = []
    for i in range(K+1):
        if not marked[i]:
            indices = [i]
            for j in range(K):
                if M[i][j] == 1:
                    indices.append(pivots[j])
            rows.append(indices)
                    
    return M, rows


def mod2(xs):
    return np.array([x % 2 for x in xs])


                
                

In [3]:
def quadratic_sieve(n, verbose=False):
    
    B = math.ceil(math.exp(math.sqrt(np.log(n) * np.log(np.log(n)))))
    
    if verbose: print("B =", B)
    
    x = round(math.sqrt(n))

    
    smooths = []
    
    S = []
    
    b_primes = [i for i in range(2, B + 1) if prime(i)]
    K = len(b_primes)
    
    
    
    ## Trying different x values
    
    print("sieving")
    
    widgets = ['b-smooth count: ', Percentage(), ' ', Bar(marker='x',left='[',right=']'),
           ' ', ETA(), ' ', FileTransferSpeed()] #see docs for other options

    pbar = ProgressBar(widgets=widgets, maxval=K+1)
    pbar.start()
    while len(smooths) <= K:
        y = (x**2) % n
        factors = factorint(y)
        if max(factors.keys()) <= B: #is b-smooth
            
            exps = np.zeros(len(b_primes))
            for f in factors:
                exps[b_primes.index(f)] = factors[f] 
                
            if verbose: print("x =", x, ": ", exps)
            S.append((x, x**2 - n))
            smooths.append(exps)
        x += 1
        pbar.update(len(smooths)) #this adds a little symbol at each iteration
    
    pbar.finish()
    
    #print(np.array(smooths))
    ## Gaussian elimination ....
    print("matrixing")
    M = np.array([mod2(s) for s in smooths])
    
    reduced_M, rows = reduce_matrix(M)
    
    #rows[0] contains the indices of M that sum to zero
    
    x = 1
    for r in rows[0]:
        x *= S[r][0]
    x = x %n
    
    y = 1
    for j in range(K):
        p = sum([smooths[r][j] for r in rows[0]])/2
        y *= b_primes[j] ** p
    y = y % n
    
    d = gcd(x-y, n)
    
    print(n, "=", d, "*", n/d)
    assert n % d == 0
    
    return d
        
        
    
    
quadratic_sieve(539873, verbose=False)
#16921456439215439701

b-smooth count: 100% [xxxxxxxxxxxxxxxxxxxxxxxxxxxxx] Time:  0:00:00  11.7 KiB/s


sieving
matrixing
539873 = 277.0 * 1949.0


277.0